# 그냥 일반적으로 널리 알려진 사실 끄적...

## R

`1` 통계분석을 위해 만들어진 언어

* 1993년 뉴질랜드 오클랜드 대학의 통계학과 교수인 `로버트 젠틀맨`과 `로스 이나카`이라는 분이 만들었음(정말, 고마운 분들)

`2` 통계분석과 시각화 측면에서는 python보다 좋다고 알려짐

`3` 또한, 애초에 프로그래밍을 처음 배우는 사람도 손쉽게 배울 수 있도록 만들어짐(만들 당시 개발 목표)

## python

`1` python은 1991년 귀도 반 로섬이 크리스마스에 연구실 출근했다가 문 닫혀서 심심풀이로 만든 언어

`2` 코미디 프로그램인 몬티 파이썬의 날아다니는 써커스 `Monty Python's Flying Circus`에서 python 이름을 따옴

`3` 그리고, 데이터 사이언스에서 초기에는 R이 우세했으나 Python이 현재는 앞지름

* 사실 앞질른 것은 맞는데, 요즘은 그런지 잘 모르겠음, 평가 기준도 애매함.

## 일반적으로 알려진 사실에 대한 내 생각

`1` 음... 일단 데이터 사이언스라는 직무를 보았을 때 numpy, pandas, tensorflow의 등장으로 python이 훨씬 편해진 것은 맞음

`2` 그러나! tidyverse, tidymodel이라는 R의 필살 패키지의 등장으로 그 장벽을 어느 정도 허물은 것 같다.

`3` 개인적인 경험

* 파이프 연산자 `%>%`는 진짜 너무 편함, python의 백엔드 연산처럼 너무 편하게 쓸 수 있음

* R이 시각화가 더 좋다고 하는데, plotly가 ggplot2보다 훨씬 코드짜기 편하고 그래프 가독성, 인터랙티브 측면에서도 훨씬 좋은 것 같음

* 대용량 데이터를 읽어드릴 때 `python`은 `pandas`를 이용해서 읽어오는데, 백만 단위가 넘어가면 상당히 오래걸림

* `R`의 `dplyr`, `data.frame` 패키지를 이용하면 이러한 데이터들도 진짜 빨리 읽어올 수 있음!

`4` 결론

* 음... 본인에 입맛에 맞는 걸, 그때그때 적절히 사용하면 될 것 같음

* 난 아직도, 데이터 50만건만 넘어가도 R로 데이터 전처리하니까...

* 그리고 데이터 분석, 모델링이라는 게 결국 전처리가 95%는 차지하는 것 같음

* 즉, 두 언어를 비교할 때, 전처리 패키지를 비교하는게 맞다고 생각한다.

* AI 모델 설계할 떄도 아직은 R의 `tidymodel` 패키지를 안 다루어 보았지만, `tidyverse`를 다루어 생각해보면 또 압도적이지 않을까라는 생각이 들음

* 오늘 그래서, 두 언어에 쓰이는 데이터 로드, 전처리를 코드 길이, 속도 측면에서 비교해보고 싶음

***

# 비교

## 샘플데이터 생성

`-` 비교를 위해 가짜 데이터를 만들어 보자(백만개만)

In [3]:
import pandas as pd
import numpy as np

In [24]:
X = np.random.random((1000000,10))
X = pd.DataFrame(X)

name = ["X"+str(i) for i in range(10)]
X.columns = name

In [26]:
X.to_csv("X.csv",index = False)

## 데이터 로드

### python

In [32]:
import time

In [33]:
start = time.time()
df = pd.read_csv("X.csv")
end  = time.time()

In [35]:
total = end-start
total

1.315992832183838

In [36]:
print(f"데이터 로드 시간(python) : {total}")

데이터 로드 시간(python) : 1.315992832183838


### R

In [25]:
library(data.table)
library(tidyverse)

In [26]:
start <- Sys.time()
df <- fread("X.csv")
end <- Sys.time()

In [27]:
total = end-start
print(paste("데이터 로드 시간(R) : ",total))

[1] "데이터 로드 시간(R) :  0.220276832580566"


* 오, 대략 `6배` 가량 차이남!

## 데이터 전처리(filter,select)

`-` 저장한 데이터에서 `x1` ~ `x5` 까지만 컬럼을 선택하고, `x1`이 0보다 큰 데이터만 추출해보자 

### python

In [1]:
import pandas as pd
import time

In [2]:
df = pd.read_csv("X.csv")

In [3]:
start = time.time()

select_col = ["X1","X2","X3","X4","X5"]

df[select_col].loc[df.X1>0,:]

end  = time.time()

In [4]:
total = end-start
print(f"데이터 전처리 시간(python) : {total}")

데이터 전처리 시간(python) : 0.03211045265197754


### R

In [3]:
library(data.table)
library(tidyverse)

df <- fread("X.csv")

In [4]:
start <- Sys.time()
df = df %>% select(X1,X2,X3,X4,X5)  %>% 
            filter(X1 > 0)
end <- Sys.time()

In [5]:
total = end-start
print(paste("데이터 전처리 시간(R) : ",total))

[1] "데이터 전처리 시간(R) :  0.0224010944366455"


`-` 전처리도 미세하게 `R이 더 빠르다.`

`-` 심지어 `코드 가독성`도 R이 더 좋은 것 같음

***

## 다중회귀모형 적합

$$x_5 = \beta_1 x_1+\beta_2 x_2+\beta_1 x_3+\beta_1 x_4+\beta_1 x_5$$

### python

In [7]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import time

In [2]:
df = pd.read_csv("X.csv")

In [3]:
df = df[["X1","X2","X3","X4","X5"]]

In [5]:
x5 = df["X5"]
X = df.drop("X5", axis = 1)

In [8]:
start = time.time()

model = LinearRegression()

model.fit(X,x5)

x5_pred = model.predict(X)

end = time.time()

In [10]:
total = end-start

In [11]:
print(f"모형 적합 및 예측 시간(python) : {total}")

모형 적합 및 예측 시간(python) : 0.1069481372833252


### R

In [2]:
library(data.table)
library(tidyverse)

In [3]:
df <- fread("X.csv")
df = df %>% select(X1,X2,X3,X4,X5)  %>% 
            filter(X1 > 0)

In [21]:
start <- Sys.time()

model <- lm(X5~. ,data = df)

x5_pred <- predict(model, df)

end <- Sys.time()

In [22]:
total = end - start
print(paste("모형 적합 및 예측 시간(R) : ",total))

[1] "모형 적합 및 예측 시간(R) :  0.293954849243164"


* 간단한 회귀모형 적합에서는 `python`이 `3배` 가량 더 빨랐다.

# summary

`1` 실제로 이렇게 비교해 본 경험은 오늘이 처음이다.

`2` 일단 데이터 로드, 전처리 부분에서는 R이 각각 `6배, 1.5배` 정도 빨랐음

`3` 모델링에서는 `python`이 3배 더 빨랐음

`4` 코드 가독성 측면에서는 R이 훨씬 더 좋았음.

`5` 그래서? 일단 내가 `R tidymodel`을 활용하여 사이드 프로젝트를 한번 해보는 경험도 좋을 것 같음

`6`  **각각 `도메인`에서 사용하는 언어들이 다르니까.....`R`, `Python` 둘 다 잘해야 하는 것 같음...** ($\star\star$)

`7` 그리고, 작년 7월에 `ISLP`라는책이 나왔는데 python에 AI 모듈들의 단점인 결과 `report` 관련해서 `ISLP` 패키지를 이용하면 어느정도 결과 보고서도 예쁘게 나오는 것 같음!